<a href="https://colab.research.google.com/github/domschl/torch-transformer-poet/blob/main/torch_transformer_poet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torch-Transformer-Poet

Please review [ml-indie-tools](https://github.com/domschl/ml-indie-tools), a collection machine learning tools that provides support for more environment indepent code. It will access your Google Drive when using with Google Colab.

In [ ]:
!pip install -U ml-indie-tools

In [ ]:
import sys
if 'google.colab' in sys.modules:
    # from: https://github.com/pytorch/pytorch/issues/107960  (libcuda not found)
    !export LC_ALL="en_US.UTF-8"
    !export LD_LIBRARY_PATH="/usr/lib64-nvidia"
    !export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
    !ldconfig /usr/lib64-nvidia

In [ ]:
import logging
import os
import copy
import json
import time
import datetime
import random
import math
import numpy as np
from zoneinfo import ZoneInfo

import torch
import torch.nn as nn
from torch import Tensor
from torch.nn import functional as F

In [ ]:
from ml_indie_tools.env_tools import MLEnv
from ml_indie_tools.Gutenberg_Dataset import Gutenberg_Dataset
from ml_indie_tools.Text_Dataset import Text_Dataset

from ml_indie_tools.Calibre_Dataset import Calibre_Dataset
from ml_indie_tools.Folder_Dataset import Folder_Dataset

import ml_indie_tools.pytorch_meta_tools as MJ
from ml_indie_tools.train_utils import TrainUtils

In [ ]:
# Optional experimental event server to record and propagate training progress, not (yet) recommended!
# Functionality is ignored by default.
try:
    import indralib
    indra_avail = True
except Exception as e:
    indra_avail = False
if indra_avail is True:
    print("Indralib is available, trying to connect to Indrajala server for training progress reports...")

In [ ]:
logging.basicConfig(level=logging.INFO)
log = logging.Logger("Main")
log.setLevel(logging.INFO)

## Preliminary

A pytorch deep multi-head attention model for text generation following Andrej Karpathy's [video-lecture-ng](https://github.com/karpathy/ng-video-lecture/blob/master/gpt.py)

This code can use either CPU, GPU, or Apple Silicon. Google Colab is supported too, select the corresponding Colab runtime (menu: **`Runtime / Change runtime type`**)

## 0. Environment

In [ ]:
cached_batch_data = None   # Do regenerate time-consuming training data, if aleady cached.

ml_env = MLEnv(platform='pt', accelerator='fastest')
ml_env.describe()

## 1. Project configuration

In [ ]:
# project_name = 'women_writers'
# project_name='research'
project_name='neo_philosophers'
model_cpu = None
model_name=f'tr_{project_name}_v3_pt'

use_preprocessed_data = True                      # Use already tokenized data
use_existing_model_from_checkpoint = False         # Try to load checkpoint of training
use_torch_compile = False                           # Requires a modern graphics card with torch compile backend support
skip_additional_texts = False                       # Don't look for other data sources in `additional_texts.json`

if 'google.colab' in sys.modules:  # Google colab notebooks run on server that provide UTC time, we adapt logs to local time:
    local_timezone = ZoneInfo('Europe/Berlin')
else:
    local_timezone = None

# NOTICE: This will request access to Google Drive, if running on Google Colab. Google Drive is used to store snapshots
# training data. See project ml-indie-tools: https://github.com/domschl/ml-indie-tools
#
# Note: you need to allow popups in your browser for COLAB, otherwise you won't see the google-drive login box, and drive access will fail!

root_path, project_path, model_path, data_path, log_path = ml_env.init_paths(project_name=project_name, model_name=model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device("mps") if torch.backends.mps.is_available() else device

print(f"Root path (all projects) : {root_path} (This will be '.' (current dir) for local projects, and a google drive path for Colab)")
print(f"Project path             : {project_path} (Changes to the file system happen only below this project path")
print(f"Model path (snapshots)   : {model_path} (Model weights and snapshots are stored here)")
print(f"Data path (training data): {data_path} (Training data will be downloaded here)")
print(f"Log dir (tensorboard)    : {log_path} (it doesn't work to put logs on gdrive due to caching, hence local dir)")

##  2.1 Text data from Project Gutenberg

`Text_Dataset` and `Gutenberg_Dataset` classes: libraries for training,
encoding, batch generation, and formatted source display. It read some
books from Project Gutenberg and supports creation of training batches.
The output functions support highlighting to allow to compare generated
texts with the actual sources to help to identify identical (memorized)
parts.

In [ ]:
use_dark_mode=False # Set to false for white background. HTML-text-compare uses background-colorization to identify different sources. Those background colors are dependent on the theme type.

In [ ]:
token_file = os.path.join(data_path,f"{project_name}_tokens.json")
if use_preprocessed_data is True:
    if os.path.exists(token_file):
        td = Text_Dataset()
        td.load_tokenizer(token_file)
    else:
        use_preprocessed_data = False

In [ ]:
if use_preprocessed_data is False:
    cache_dir = os.path.join(data_path, 'gutenberg_cache')
    gd = Gutenberg_Dataset(cache_dir=cache_dir)

    if project_name == 'women_writers':  # sample searches
        search_spec= {
            "author": ["Emily Brontë", "Jane Austen", "Virginia Woolf"],
            "language": ["english"]
        }
        book_list=gd.search(search_spec)
    elif project_name == 'neo_philosophers':
        search_spec = {
            "author": ["Immanuel Kant", "Friedrich Nietzsche", "Wilhelm Hegel", "Arthur Schopenhauer"],
            "language": ["english", "german"]
        }
        book_list=gd.search(search_spec)
        search_spec = {
            "author": ["Plato", "Platon"],
            "title": ["Timaeus", "Critias", "Symposium"],
            "language": ["english", "german"]
        }
        book_list+=gd.search(search_spec)
        search_spec = {
            "title": ["Buddh", "Sutra"],
            "language": ["english", "german"]
        }
        book_list+=gd.search(search_spec)
    else:
        search_spec = {}
        book_list = []

    book_cnt = len(book_list)
    print(f"{book_cnt} matching books found with search {search_spec}.")

    if book_cnt > 0:
        if book_cnt<80:
            # Note: please verify that book_cnt is 'reasonable'. If you plan to use a large number of texts,
            # consider [mirroring Gutenberg](https://github.com/domschl/ml-indie-tools#working-with-a-local-mirror-of-project-gutenberg)
            book_list = gd.insert_book_texts(book_list, download_count_limit=book_cnt)
        else:
            logging.error("Please verify your book_list, a large number of books is scheduled for download. ABORTED.")

        for i in range(len(book_list)):
            if 'author' not in book_list[i]:
                book_list[i]['author']='unknown'
            print(f"{i}: {book_list[i]['title']} - {book_list[i]['author']}, {book_list[i]['ebook_id']}")

        if project_name == 'women_writers':
            select = ("Bennett", "1342", "5670", "1245", "161", "141", "121", "105", "Susan", "Wuthering", "Emma", "Voyage")  # List unique single-words from title or ebook_id to select a given book
            sub_book_list = [book_list[i] for i in range(len(book_list)) if not set([book_list[i]['ebook_id']]+book_list[i]['title'].split(' ')).isdisjoint(set(select))]
        else:
            sub_book_list = book_list

        print("Using:")
        for i in range(len(sub_book_list)):
            if 'author' not in sub_book_list[i]:
                sub_book_list[i]['author']='unknown'
            print(f"{i+1}: {sub_book_list[i]['title']} - {sub_book_list[i]['author']}")

        td = Text_Dataset(sub_book_list)
    else:
        td = Text_Dataset()()

## 2.2 Additional training material from folders or Calibre library

This looks for a file `additional_texts.json` in the `project_path` as shown above.

```json
{
  "local_texts": [["/some/directory/that/contains/texts", [".txt", ".md", ".org", ".py"]],
  "calibre": ["/home/myuser/Calibre Library", []]
}
```

If the folder(s) defined in `local_texts` contain text files with default endings `.txt`, `.md`, `.org`, or `.py` (can be configured), they are added to the training data. Folders are searched recursively.

If the path defined in `calibre` contains a Calibre database, all text files (`.txt` only) within that library are added to the training data. The list argument can contain search-specs (see ml-indie-tools, calibre_dataset) to qualify which books to import, e.g. [{"tags": ["philosophy"]}] would import all books that are tagged with "Philosophy" within calibre.

In [ ]:
if use_preprocessed_data is False and skip_additional_texts is False:
    additional = os.path.join(project_path, "additional_texts.json")
    print(f"Looking for description of additional sources in {additional}")
    if os.path.exists(additional) is True:
        with open(additional, 'r') as f:
            add_desc = json.load(f)
            if 'local_texts' in add_desc:
                fd = Folder_Dataset()
                for text_path, qualifier in add_desc['local_texts']:
                    if not isinstance(qualifier, list):
                        qualifier = [qualifier]
                    print(f"Loading texts from {text_path} using extension restriction {qualifier}")
                    fd.load_index(text_path, use_aliases=False, min_file_size=2048, file_extensions=qualifier)
                td.load_texts(fd.records[:10000])
            if 'calibre' in add_desc:
                cal_path, specs = add_desc['calibre']
                if os.path.exists(cal_path):
                    print(f"Loading text from calibre at {cal_path}")
                    cd = Calibre_Dataset(cal_path)
                    cd.load_index(max_file_size=100000000) 
                    if specs is not None and len(specs)!=0:
                        ls = cd.search(specs)
                        td.load_texts(ls)
                    else:
                        td.load_texts(cd.records[:1000])

## 2.3 Tokenize data

In [ ]:
if use_preprocessed_data is False:
    MAX_TOKENS = 32768  # This becomes vocab_size
    MAX_NGRAM_LEN = 5   # Max length of a token
    CHUNK_SIZE = 500000 # Split larger texts in chunks, if not None

    print("")
    print(f"Starting tokenizer with token length from 1..{MAX_NGRAM_LEN} with a max of {MAX_TOKENS} unique tokens,")
    print("this can take considerable time...")

    # Better tested NGRAM tokenizer:
    # td.init_tokenizer(tokenizer='ngram', max_ngrams=MAX_NGRAM_LEN, max_tokens=MAX_TOKENS) 
    # or alternative 'BYTEGRAM' (more experimental, can encode arbitrary UTF-8)
    # td.init_tokenizer(tokenizer='bytegram', max_ngrams=MAX_NGRAM_LEN, max_tokens=MAX_TOKENS, chunk_size=CHUNK_SIZE)
    td.init_tokenizer(tokenizer='bytegram', max_ngrams=MAX_NGRAM_LEN, max_tokens=MAX_TOKENS, chunk_size=CHUNK_SIZE)
    td.save_tokenizer(token_file)

In [ ]:
tok_tests = ["Good morning, this is a simple test sentence for tokenization", "Guten Morgen, dies is ein einfach Testsatz zur Aufteilung in Satzbestandteile", "སེམས་ཉིད་ངལ་བསོ་རྒྱུད་"]
for test in tok_tests:
    enc = td.encode(test)
    dec = td.decode(enc)
    if dec != test:
        print(f"Tokenizer failed for: {test} -> {dec}")
    else:
        r = len(enc)/len(test)*100.0
        print(f"Tokenizer: {test}({len(test)}) -> {enc}({len(enc)}) OK, compressed size: {r:.2f}%")

## 3. Model metadata

In [ ]:
params = None
updatable_keys=['learning_rate', 'batch_size', 'current_epoch', 'current_loss',
                 'sample_every_n_iterations', 'sample_size', 'save_every_n_iterations', 'max_iterations']
attn_layers = 4
dims = 256
sequence_length = 256

params = { # Multi-head self-attention
        'meta_name_template': '{mhsa_layers}x{heads}x{units}x{vocab_size}',

        'mhsa_layers': attn_layers,
        'heads': 16,
        'vocab_size': td.get_unique_token_count(),
        'sequence_len': sequence_length,
        'dropout': 0.1,
        'embedding_size': dims,
        'test_iterations': 100,  # number of epocs for loss estimation
        'use_recur': False,
        'share_weights': False,

        'batch_size': 48,      
        'learning_rate': 1e-4,   # None: Set in dependence of graphics hw
        'lr_schedule': True,
        'lr_min': 2e-4,
        'lr_max': 1e-3,
        'warmup': 2000,
        'decay': 100000,
        'grad_clip': 0.3,

        'sample_every_n_iterations': 2048,
        'sample_size': 128,
        'save_every_n_iterations': 4096,

        'max_iterations': 100000000  # maximum number of training iterations
    }

model_file_path = MJ.get_model_filename(model_path)
if use_existing_model_from_checkpoint is True:
    params = MJ.load_model_metadata_from_checkpoint(params, updatable_keys, model_file_path, device=device, log=log) # torch.device('cpu'))
if params == None or use_existing_model_from_checkpoint is False:
    use_existing_model_from_checkpoint = False
# print(params)

## 4. Batch handling

In [ ]:
joint_training=0
td.init_getitem(sample_type='encoded', sample_length=params['sequence_len']+1+joint_training, content_stepping=1)
num_records = len(td)
print(f"{num_records} records")

In [ ]:
def get_sample_sub_batch(sample_batch, batch_size, sub_index=0):
    joint_training=0
    for i in range(batch_size):
        Xi = sample_batch[sub_index:-1-joint_training+sub_index]
        yi = sample_batch[sub_index+1:]
        if i==0:
            # smpX=np.array(Xi, dtype=np.float32)
            smpX=np.array(Xi, dtype=np.int32)
            smpy=np.array(yi, dtype=np.int32)
        else:
            # smpX = np.vstack((smpX, np.array(Xi, dtype=np.float32)))
            smpX = np.vstack((smpX, np.array(Xi, dtype=np.int32)))
            smpy = np.vstack((smpy, np.array(yi, dtype=np.int32)))
    return np.array(smpX), np.array(smpy)

def get_sample_batch(td, batch_size):
    sample_batch = td.get_random_item()
    return get_sample_sub_batch(sample_batch, batch_size)

In [ ]:
num_batches = num_records // params['batch_size']
print(f"num_batches = {num_batches}")

In [ ]:
x, y = get_sample_batch(td, 2)
x.shape, y.shape

In [ ]:
sample_data = None

def get_torch_subbatch(td, batch_size, device, split=None, sub_index=0):
    global sample_data
    if sub_index==0:
        sample_data = td.get_random_item()
    x, y = get_sample_sub_batch(sample_data, batch_size, sub_index)
    tx = torch.tensor(x, dtype=torch.long).to(device)
    tx.requires_grad = False
    ty = torch.tensor(y, dtype=torch.long).to(device)
    ty.requires_grad = False
    return tx, ty

def get_torch_batch(td, batch_size, device, split=None):
    x, y = get_sample_batch(td, batch_size)
    tx = torch.tensor(x, dtype=torch.long).to(device)
    tx.requires_grad = False
    ty = torch.tensor(y, dtype=torch.long).to(device)
    ty.requires_grad = False
    return tx, ty

def get_zero_state(batch_size, sequence_len, hidden_size, device):
    zstate = torch.zeros(batch_size, sequence_len, hidden_size, device=device)
    zstate.requires_grad = False
    return zstate

## 5. Loss and training helpers

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000, device=None):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model, device=device)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
        
class MultiHeadSelfAttentionWithMemory(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding_size,
        sequence_len,
        num_heads,
        num_layers,
        dropout=0.1,
        use_recur=False,
        share_weights=True,
        device=None,
    ):
        super().__init__()
        if device is None:
            raise ValueError(
                "Device is None at MultiHeadSelfAttentionWithMemory"
            )
        self.device = device
        self.sequence_len = sequence_len
        self.use_recur = use_recur
        context_sub_layers = num_layers // 2
        self.context_sub_layers = context_sub_layers
        dims = embedding_size
        self.dims = dims
        # each token directly reads off the logits for the next token from a lookup table
        self.position_embedding_table = nn.Embedding(
            sequence_len, embedding_size, device=device
        )

        if share_weights is True:
            self.embedding = nn.Embedding(vocab_size, embedding_size)
            self.out_proj = nn.Linear(embedding_size, vocab_size, bias=False)
            self.embedding.weight = self.out_proj.weight  # torch.nn.Parameter(self.out_proj.weight.transpose(1,0))
            self.out_proj = self.out_proj.to(device)
            self.embdding = self.embedding.to(device)
        else:
            self.embedding = nn.Embedding(vocab_size, embedding_size, device=device)
            self.out_proj = nn.Linear(embedding_size, vocab_size, device=device)


        if use_recur is True:
            self.rec=nn.RNN(dims, dims, num_layers=3, batch_first=True, device=device)
            self.lq=nn.Linear(dims, dims, device=device)
            self.lk=nn.Linear(dims, dims, device=device)
            self.lv=nn.Linear(dims, dims, device=device)
            self.lnorm=nn.BatchNorm1d(dims, device=device)
            self.sm = nn.Softmax(dim=2)
            self.proj1 = nn.Linear(dims, dims, device=device)
            self.sm2 = nn.Softmax(dim=2)
            encoder_layer = nn.TransformerEncoderLayer(d_model=dims, nhead=num_heads, dim_feedforward=dims*4, dropout=dropout, batch_first=True, device=device)
            self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=context_sub_layers)
            encoder_layer2 = nn.TransformerEncoderLayer(d_model=dims, nhead=num_heads, dim_feedforward=dims*4, dropout=dropout, batch_first=True, device=device)
            self.transformer2 = nn.TransformerEncoder(encoder_layer2, num_layers=num_layers - context_sub_layers)
        else:
            encoder_layer = nn.TransformerEncoderLayer(d_model=dims, nhead=num_heads, activation="gelu", dim_feedforward=dims*4, dropout=dropout, batch_first=True, device=device)
            self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def init_weights(self) -> None:
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.out_proj.bias.data.zero_()
        self.out_proj.weight.data.uniform_(-initrange, initrange)

    def forward(self, idx):
        B, D = idx.shape
        tok_emb = self.embedding(idx)
        # idx and targets are both (B,D) tensor of integers
        # tok_emb = self.token_embedding_table(idx)  # (B,D,C)

        # XXX: move to init, make not trainable:
        if self.device is None:
            pos_emb = self.position_embedding_table(torch.arange(self.sequence_len))
        else:
            pos_emb = self.position_embedding_table(
                torch.arange(D, device=self.device)
            )  # (D,C)

        x = tok_emb + pos_emb  # (B,D,C)
        
        # x = self.pos_encoder(x) 
        x_mask = nn.Transformer.generate_square_subsequent_mask(D).to(self.device)
        x = self.transformer(x, x_mask)

        if self.use_recur is True:
            skip = x
            x = self.rec(x)[0] + x
            xk = self.lk(x).permute((0,2,1))
            xv = self.lv(x)
            xq = self.lq(x)
            xqk = torch.matmul(xq, xk)
            sm = self.sm(xqk)/math.sqrt(D)
            att = torch.matmul(sm, xv)
            x = self.lnorm(att)
            x = self.sm2(self.proj1(x))
            x = x + skip    
            x = self.transformer2(x, x_mask)
            
        logits = self.out_proj(x)
        return logits

    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """Generate new tokens given a context

        Note: for apple MPS, top_k is limited max 16 vor older torchs! ((01/2023) implementation limitation)
        See: https://github.com/pytorch/pytorch/issues/78915
        Solved in: https://github.com/pytorch/pytorch/pull/94639 (03/2023)

        :param idx: the context (B,T) tensor of indices
        :param max_new_tokens: the maximum number of tokens to generate
        :param temperature: the temperature to use for sampling
        :param top_k: the number of top tokens to consider
        """
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last sequence_len tokens
            idx_cond = idx[:, -self.sequence_len :]
            # print(idx_cond.shape)
            # get the predictions
            logits = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float("Inf")
            # apply temperature
            if temperature != 1.0 and temperature > 0.0:
                logits = logits / temperature
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


In [ ]:
print("creating model...")
try:
    # Colab + torch 2 -> lots of garbage.
    if model is not None:
        del model
except:
    pass


model = MultiHeadSelfAttentionWithMemory(vocab_size=params['vocab_size'], embedding_size=params['embedding_size'],
                                       sequence_len=params['sequence_len'],
                                       num_heads=params['heads'], num_layers=params['mhsa_layers'], dropout=params['dropout'],
                                       use_recur=params['use_recur'], share_weights=params['share_weights'], device=device)

optimizer = torch.optim.AdamW(model.parameters(), lr=params['learning_rate'])

model = model.to(device)
if use_existing_model_from_checkpoint is True:
    params_load = MJ.load_checkpoint(params, model, optimizer, file_path=model_file_path, updatable_keys=updatable_keys, device=device, log=log) # torch.device("cpu"))
    if params_load is not None:
        params = params_load
model = model.to(device)
for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)

if use_torch_compile is True:
    if device == 'cuda':
        print("Compiling...")
        model = torch.compile(model)
        print("Compile ok.")
        try:
            torch.set_float32_matmul_precision('high')
        except:
            print("Seems no tensor cores for that.")
    # elif str(device) == 'mps':
    #     print("Compiling...")
    #     model = torch.compile(model)
    #     print("Compile ok.")

if 'current_epoch' in params:
    ep = params['current_epoch']
else:
    ep=0
if 'current_loss' in params:
    ls = params['current_loss']
else:
    ls=0

if ep==0 and ls==0:
    start_iter = 0
else:
    start_iter = ep
    current_loss = ls

# print the number of parameters in the model
print(model)
print(sum(p.numel() for p in model.parameters()) / 1e6, "M parameters")

In [ ]:
@torch.no_grad()
def estimate_loss(device):
    # XXX: this does take data for train and val from SAME pool!
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(params['test_iterations'])
        for k in range(params['test_iterations']):
            # if k % (params['test_iterations']/10 + 1) == 0:
            #     print(".", end="", flush=True)
            X, Y = get_torch_batch(td, params['batch_size'], device, split)
            logits = model(X)
            loss = get_loss(logits, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    print("\r", end="", flush=True)
    mloss = (out['train']+out['val'])/2.0
    return mloss

def generate_sample(td, device, prompt=' ', toks=100, state=None, temperature=1.0, top_k=None, pad=True):
    # generate from the model
    # context = torch.zeros((1, 1), dtype=torch.long, device=device)
    model.eval()
    if pad is True:
        while len(prompt)<params['sequence_len']*4:
            if len(prompt)==params['sequence_len']*4-1:
                prompt = '\n' + prompt
            else:
                prompt = ' ' + prompt
    context = torch.tensor([td.encode(prompt)]).to(device)
    answer = model.generate(context, max_new_tokens=toks, temperature=temperature, top_k=top_k)
    txt = td.decode(answer[0].tolist())
    # Identify memorisation of text by highlighting verbatim quotes from sources
    # that are longer than 10 chars. HTML colorcoded output for source identification:
    td.source_highlight(txt, min_quote_size=10, dark_mode=False, display_ref_anchor=False)
    model.train()
    return txt


In [ ]:
# @torch.jit.script
# @torch.compile
criterion = nn.CrossEntropyLoss()

def get_loss(logits, yb):
    output_flat = logits.reshape(-1, params['vocab_size'])
    # output_flat = logits.view(-1, params['vocab_size'])
    # print(output_flat.shape)
    ybr = yb.reshape(-1)
    # print(ybr.shape)
    loss = criterion(output_flat, ybr)
    return loss
    
def do_train_step(xb, yb, device, state=None):
    model.train()
    logits = model(xb)
    loss = get_loss(logits, yb)
    
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), params['grad_clip']).cpu()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item(), norm

In [ ]:
def start_tu_session():
    if indra_avail is True:
        with open('indra_creds.json', 'r') as f:
            creds = json.load(f)
            tu = TrainUtils(indra_server_profile_name='default', username=creds['username'], password=creds['password'])
    else:
        tu = TrainUtils()
    tu.train_session_start(model_name=model_name, model_description="Torch-poet tests", model_version=1, model_params=params, indra_subdomain="torch_poet/first_tests/1", status_string_size=110)
    return tu

In [ ]:
def lr_schedule(optim, n_iter, warmup, max_lr, decay, min_lr):
    if n_iter<warmup and warmup>0:
        lr = (n_iter+1)/warmup*max_lr
    elif n_iter<warmup+decay and decay>0:
        i = n_iter-warmup
        lr = (decay-i)/decay*(max_lr-min_lr)+min_lr
    else:
        lr = min_lr

    for g in optim.param_groups:
        g['lr'] = lr
    return lr


In [ ]:
def train(train_utils):
    global start_iter
    dt0 = time.time()
    sdt = datetime.datetime.now(tz=local_timezone).strftime("%Y-%m-%d %H:%M:%S")
    print(f"training, start at {sdt}...")
    gen_id = 0
    last_print=0
    iter_bench = 1
    tu = train_utils
    lr = params['learning_rate']
    # current_loss = estimate_loss(device)
    inputs = ["What is the difference between good and evil? The difference ", "How did everything come into existence? The origin ", "What was at the beginning of time? Time itself ", "How are physics, quantum-mechanics and consciousness related? The relation between ", "How to attain complete self-awareness? Complete ", "What is the nature of reality? The nature ", "How be a good human being? A human "]
    for iter in range(start_iter, params['max_iterations']):
        # every once in a while evaluate the loss on train and val sets
        if (iter + 1) % params['sample_every_n_iterations'] == 0 or iter == params['max_iterations'] - 1:
            dt = time.time()
            print(f"\rloss eval", end="", flush=True)
            current_loss = estimate_loss(device)
            print(
                f"step {iter+1}: train loss {current_loss:.4f}, time {(dt-dt0)/iter_bench:.3f} sec/iter                       "
            )
            iter_bench = 1
            sdt = datetime.datetime.now(tz=local_timezone).strftime("%Y-%m-%d %H:%M:%S")
            print(f"Sample at {sdt}:", flush=True)
            for temperature in [0.75]:
                print(f"--------temperature: {temperature} ---------")
                prompt = inputs[gen_id%len(inputs)]
                print(f"Prompt: {prompt}")
                generate_sample(td=td, device=device, prompt=prompt, toks=params['sample_size'], temperature=temperature, top_k=4)
            print("-------------------------------------------")
            gen_id += 1
            dt0 = time.time()
    
        if params['lr_schedule'] is True:
            lr = lr_schedule(optimizer, iter, params['warmup'], params['lr_max'], params['decay'], params['lr_min'])
    
        xb, yb = get_torch_batch(td, params['batch_size'], device, "train")
        cur_loss, cur_norm = do_train_step(xb, yb, device=device)

        
        nt = time.time()
        if (nt-last_print)>1:
            rec = {
                'epoch': iter/num_batches,
                'batch': iter%params['sample_every_n_iterations'],
                'num_batches': params['sample_every_n_iterations'],
                'loss': cur_loss,
                'learning_rate': lr,
                'gradient_norm': cur_norm.item(),
            }
            status_string, record = train_utils.train_state(rec)
            print(status_string, end="\r")
            last_print=nt
        
        start_iter = iter
        iter_bench += 1
        if (iter+1)%params['save_every_n_iterations'] == 0:
            MJ.save_checkpoint(params, model, optimizer, iter, current_loss, file_path=model_file_path, log=log)

In [ ]:
tu = start_tu_session()
try:
    train(train_utils = tu)
except KeyboardInterrupt:
    print(f"\nTraining interrupted.")
tu.train_session_end()

In [ ]:
# for t in [0.5, 1.5]:
#     print(f"------Temperature {t}--------")
#     generate_sample(td, device, prompt="How are consciousness and quantum mechanics related?", toks=150, temperature=t, top_k=16)

In [ ]:
# Test-code below, unfinished.

In [ ]:
texts = []
enc_texts = []
for i in range(500):
    e = td[i*50000][:256]
    tx = torch.tensor([e]).to(device)
    enc_texts.append(tx)
    texts.append(td.decode(e))


In [ ]:
enc_texts[0].shape

In [ ]:
emb_text = []
cont_text = []
for et in enc_texts:
    emb_text.append(model.embedding(et))
    cont_text.append(model.context(et))

In [ ]:
emb_text[0].shape, cont_text[0].shape

In [ ]:
emb_vec = []
cont_vec = []
for i in range(len(emb_text)):
    emb_vec.append(emb_text[i][0].sum(axis=0))
    cont_vec.append(cont_text[i][0].sum(axis=0))

In [ ]:
def cos_vec(a, b):
    al = torch.sqrt(torch.dot(a,a))
    bl = torch.sqrt(torch.dot(b,b))
    an = a/al
    bn = b/bl
    return torch.dot(an,bn)

In [ ]:
for i in range(0):
    best = -10.0
    ind = -1
    for j in range(len(emb_vec)):
        if i==j:
            continue
        cos_val = cos_vec(emb_vec[i], emb_vec[j])
        if cos_val > best:
            best = cos_val
            ind = j
    # print(f"{texts[i][:20]} ->{best}: {texts[ind][:20]}")
print()        
for i in range(200):
    best = 0
    ind = -1
    for j in range(len(emb_vec)):
        if i==j:
            continue
        cos_val = cos_vec(cont_vec[i], cont_vec[j])
        if cos_val > best:
            best = cos_val
            ind = j
    print(f"{texts[i]} \n\n->{best}:\n\n {texts[ind]}")
    print("---------------------------------------------")


In [ ]:
td[200002]